Let $\vec{D} = (D_i)_{i=1}^{n}$ with $D_i = f(x_i) + E_i$ with $E_i \sim \mathcal{N}(0, \sigma^2)$, where $f$ is unknown.

Let $d$ be a realization of $D$ and $s$ be a deterministic solver that returns the parameters of a function $\hat{f}$ that approximates $f$:

$$
s(d) = \argmin_{\theta} \frac{1}{n} \sum_{i=1}^{n} \left( \hat{f}(x_i, \theta) - d_i \right)^2.
$$

Let $g$ be a function on a set of datapoints (e.g. a function that generates a synthetic dataset from a given dataset). $g$ also takes another parameter.

$$
g(d, q) = \tilde{d}
$$

Let $Q$ be a random variable with distribution $p(Q)$.

I already obtained samples $\hat{f}(x, s(g(D, q_k)))$ by simply sampling $q_k$'s from $p(Q)$ and now want to use importance sampling to obtain:

$$
p(\hat{f}(x, s(g(D, Q))) | D)
$$

No further assumptions can be made. Attempt the problem without any yapping about its diffculty.

Here is my procedure to approximate the uncertainty of the output of a NN conditioned on the training data $d$:
1. generate samples $u_k$, $k=1 \dots m$ ($u \in [0,1]$)
2. generate synthetic datasets $d_k \coloneqq g(d, u_k)$
3. train NNs $\hat{f}_k$ on those
4. evaluate those: $\hat{y}_k(x) = \hat{f}_k(x)$
5. weight samples such that correct distribution is obtained

I am not sure about the weighting process, which is why I come to you. I just know that I have to reweight because of the following thought experiment:
I know that $g$ gives synthetic datasets that can look completely different from another for large, but different $u$'s. The opposite is true for small but different $u$'s.
Imagine two different sampling strategies for $u_k$. The first gives very large $u$'s and the second very small ones. Since my NN learns these datasets, the same while hold for the predicted values $\hat{y}_k(x)$: First samling strategy -> high variance. Second samling strategy -> low variance. So two different uncertainty quantifications for the same dataset $d$.

Formalize my procedure step by step using exact mathematical notation.

But before doing so, formalize the procedure of using Bayes neural networks (BNNs) with MCMC to approximate the uncertainty of the output of a NN conditioned on the training data. Include every step in the process, as if this intended for a reader hearing the first time about BNNs.

Make sure to use consistent notation: Uppercase for random variables, f,g,h,... for functions and so on. Be extra precise and pedantic about everything.

When you are done with BNNs, let me read over it and then we may proceed with my procedure.

Let's formalize Bayesian Neural Networks and your proposed procedure for uncertainty quantification.

**Bayesian Neural Networks (BNNs) with MCMC for Uncertainty Quantification**

1. **Model Definition:** A neural network can be viewed as a function $f(x; w)$, where $x \in \mathcal{X}$ is the input, $w \in \mathcal{W}$ are the network weights, and $f: \mathcal{X} \times \mathcal{W} \to \mathcal{Y}$ maps inputs to outputs in the space $\mathcal{Y}$.  In a Bayesian setting, we treat the weights $W$ as a random variable.

2. **Prior Distribution:** We define a prior distribution $p(W)$ over the weights. This prior represents our initial beliefs about the weights before observing any data.  Common choices include Gaussian priors, e.g., $W \sim \mathcal{N}(0, \sigma^2 I)$, where $I$ is the identity matrix.

3. **Likelihood Function:** Given a dataset $d = \{(x_i, y_i)\}_{i=1}^n$ consisting of $n$ input-output pairs, we define a likelihood function $p(D|W)$, which represents the probability of observing the data given the weights.  Assuming independent and identically distributed (i.i.d.) data points and a Gaussian noise model, the likelihood can be written as:

   $$p(D|W) = \prod_{i=1}^n p(y_i|x_i, W) = \prod_{i=1}^n \mathcal{N}(y_i | f(x_i; W), \sigma_n^2)$$

   where $\sigma_n^2$ is the noise variance.

4. **Posterior Distribution:** Using Bayes' theorem, we can compute the posterior distribution $p(W|D)$, which represents our updated beliefs about the weights after observing the data:

   $$p(W|D) = \frac{p(D|W) p(W)}{p(D)}$$

   where $p(D) = \int p(D|W) p(W) dW$ is the marginal likelihood (evidence), which acts as a normalizing constant.

5. **Markov Chain Monte Carlo (MCMC) Sampling:** Since the posterior distribution is often intractable to compute directly, we use MCMC methods to draw samples from it.  A common MCMC method is the Metropolis-Hastings algorithm.  It constructs a Markov chain whose stationary distribution is the target posterior distribution $p(W|D)$.  The algorithm proceeds as follows:

    a. Initialize the weights $w^{(0)}$.
    b. For $t = 1, 2, \dots, T$:
    i. Propose a new set of weights $w'$ from a proposal distribution $q(w'|w^{(t-1)})$.
    ii. Calculate the acceptance ratio:

    $$\alpha = \min\left(1, \frac{p(D|w')p(w')q(w^{(t-1)}|w')}{p(D|w^{(t-1)})p(w^{(t-1)})q(w'|w^{(t-1)})}\right)$$

    iii. Accept the proposed weights $w'$ with probability $\alpha$, i.e., set $w^{(t)} = w'$. Otherwise, set $w^{(t)} = w^{(t-1)}$.

6. **Predictive Distribution:**  After obtaining $T$ samples $\{w^{(t)}\}_{t=1}^T$ from the posterior, we can approximate the predictive distribution for a new input $x^*$:

   $$p(Y^*|x^*, D) \approx \frac{1}{T} \sum_{t=1}^T p(Y^*|x^*, w^{(t)}) = \frac{1}{T} \sum_{t=1}^T \mathcal{N}(Y^* | f(x^*; w^{(t)}), \sigma_n^2)$$

   This predictive distribution provides both a point estimate (e.g., the mean) and uncertainty quantification (e.g., the variance or credible intervals).

Now, let's proceed with your procedure.  Please let me know when you're ready.


6. **Predictive Distribution:** The goal is to predict the output $Y^*$ for a new input $x^*$ given the observed dataset $D$. In the Bayesian framework, this involves computing the predictive distribution $p(Y^*|x^*, D)$. This distribution incorporates the uncertainty over the weights $W$ by marginalizing over the posterior distribution $p(W|D)$:

   $$p(Y^*|x^*, D) = \int p(Y^*|x^*, W) p(W|D) dW$$

   Here, $p(Y^*|x^*, W)$ represents the likelihood of observing the output $Y^*$ given the input $x^*$ and a specific set of weights $W$.  As before, we typically assume a Gaussian noise model:

   $$p(Y^*|x^*, W) = \mathcal{N}(Y^* | f(x^*; W), \sigma_n^2)$$

   Since the integral in the predictive distribution is often intractable, we approximate it using the MCMC samples obtained in the previous step.  Specifically, we use the Monte Carlo estimator:

   $$p(Y^*|x^*, D) \approx \frac{1}{T} \sum_{t=1}^T p(Y^*|x^*, w^{(t)})$$

   where $\{w^{(t)}\}_{t=1}^T$ are the $T$ samples drawn from the posterior distribution $p(W|D)$ using MCMC. Substituting the Gaussian likelihood, we get:

   $$p(Y^*|x^*, D) \approx \frac{1}{T} \sum_{t=1}^T \mathcal{N}(Y^* | f(x^*; w^{(t)}), \sigma_n^2)$$

   This approximation represents a mixture of Gaussian distributions, each centered around the network's prediction $f(x^*; w^{(t)})$ with variance $\sigma_n^2$ for each sampled weight vector $w^{(t)}$.

   From this approximated predictive distribution, we can extract various quantities for uncertainty quantification:

   * **Mean Prediction:**  The mean of the predictive distribution provides a point estimate for $Y^*$:
     $$E[Y^*|x^*, D] \approx \frac{1}{T} \sum_{t=1}^T f(x^*; w^{(t)})$$

   * **Variance:** The variance of the predictive distribution quantifies the uncertainty in the prediction:
     $$Var[Y^*|x^*, D] \approx \frac{1}{T} \sum_{t=1}^T \left[ (f(x^*; w^{(t)}) - E[Y^*|x^*, D])^2 + \sigma_n^2 \right]$$

   * **Credible Intervals:**  A $(1-\alpha)\%$ credible interval represents an interval within which the true value of $Y^*$ lies with $(1-\alpha)\%$ probability.  These intervals can be estimated from the quantiles of the samples $\{f(x^*; w^{(t)})\}_{t=1}^T$.  For example, a 95% credible interval is defined by the 2.5th and 97.5th percentiles of the samples.


This detailed explanation clarifies how the predictive distribution is approximated using MCMC samples and how various uncertainty measures can be derived.  Now we can move on to your proposed procedure when you're ready.


Alright, let's formalize your procedure for approximating the uncertainty of a neural network's output conditioned on the training data.

**Your Proposed Procedure**

1. **Generate Samples:** Generate $m$ independent and identically distributed (i.i.d.) samples $U_k$ from a uniform distribution over the unit interval, i.e., $U_k \sim \mathcal{U}(0, 1)$ for $k = 1, \dots, m$.

2. **Generate Synthetic Datasets:**  Apply a function $g$ to the original dataset $d$ and each sample $U_k$ to create $m$ synthetic datasets $d_k$:

   $$d_k = g(d, U_k), \quad k = 1, \dots, m$$

   The function $g: \mathcal{D} \times [0, 1] \to \mathcal{D}$ maps a dataset and a uniform random variable to a new synthetic dataset.  The exact form of $g$ will determine how the synthetic datasets are generated.  This is where your intuition about small $u$ values leading to similar datasets and large $u$ values leading to dissimilar datasets comes into play.

3. **Train Neural Networks:**  Train $m$ neural networks $\hat{f}_k(x; w_k)$, where $w_k$ are the weights of the $k$-th network, on each of the synthetic datasets $d_k$.  This results in $m$ different trained networks:

   $$\hat{f}_k = \text{train}(d_k), \quad k = 1, \dots, m$$

   Here, $\text{train}(\cdot)$ represents the training process, which could be any standard training algorithm (e.g., stochastic gradient descent).

4. **Evaluate Networks:** For a new input $x$, evaluate each of the $m$ trained networks to obtain $m$ predicted outputs:

   $$\hat{y}_k(x) = \hat{f}_k(x; w_k), \quad k = 1, \dots, m$$

5. **Weight Samples:**  Compute weights $\omega_k$ for each prediction $\hat{y}_k(x)$ such that the weighted average of the predictions approximates the true predictive distribution conditioned on the original dataset $d$. This is the crucial step where we need to determine the appropriate weighting scheme.

   The goal is to find weights $\omega_k(U_k)$ such that:

   $$ p(Y|x, D) \approx \sum_{k=1}^m \omega_k(U_k) \delta(Y - \hat{y}_k(x)) $$

   where $\delta(\cdot)$ is the Dirac delta function.

To determine $\omega_k(U_k)$, we have to compute $p(Y|x, D)$ by marginalization. Would you like me to perform this calculation now?

We want to approximate $p(Y|x, D)$, the true predictive distribution given the original dataset $D$.  Your method generates synthetic datasets $d_k = g(D, U_k)$ using the transformation function $g$ and random variables $U_k$.  The key is to understand how the distribution of the synthetic datasets relates to the true data distribution.

Let's denote the true data distribution as $p(D)$. We can think of your process as generating samples from a distribution $p_g(D')$ of datasets $D'$, induced by $g$ and the distribution of $U$.  Formally, this is given by:

$$p_g(D') = \int p(U) \delta(D' - g(D, U)) dU$$

where $\delta$ is the Dirac delta function.  Ideally, we would like $p_g(D')$ to be close to $p(D)$.  However, this is generally not the case, especially if $g$ introduces significant distortions.

Now, consider the predictive distribution given a synthetic dataset $d_k$:

$$p(Y|x, d_k) \approx \mathcal{N}(Y | \hat{f}_k(x), \sigma_{n,k}^2)$$

where we're approximating the predictive distribution with a Gaussian centered at the network prediction $\hat{f}_k(x)$ with a variance $\sigma_{n,k}^2$ (which could be estimated during training).

To connect this back to the true predictive distribution, we can think of marginalizing over the distribution of synthetic datasets:

$$p(Y|x, D) = \int p(Y|x, D') p(D'|D) dD'$$

This is where the weighting comes in.  We want to approximate this integral using our samples $d_k$.  The weights $\omega_k(U_k)$ should ideally be proportional to the importance weights:

$$\omega_k(U_k) \propto \frac{p(d_k|D)}{p_g(d_k)} = \frac{p(g(D, U_k)|D)}{p_g(g(D, U_k))}$$

These importance weights correct for the discrepancy between the distribution of synthetic datasets $p_g(D')$ and the true posterior distribution over datasets given the original data, $p(D'|D)$.  This posterior is generally intractable, which makes calculating these weights directly difficult.

**Approximations and Practical Considerations:**

Since computing $p(d_k|D)$ and $p_g(d_k)$ is usually intractable, we need to consider approximations. Here are some possible directions:

* **Assume $p(D'|D) \approx p_g(D')$:**  If we assume that the distribution of synthetic datasets is a reasonable approximation to the true posterior over datasets, then the weights can be simplified to $\omega_k \approx 1/m$, which corresponds to a simple average.  This is the simplest approach but might not be accurate if $g$ introduces significant biases.

* **Density Estimation:**  We could try to estimate $p_g(d_k)$ using density estimation techniques. This is generally challenging in high dimensions.

* **Variational Inference:**  We could try to learn a variational approximation to $p(D'|D)$ and use this to compute the importance weights.

* **Metropolis-Hastings:**  We could potentially adapt Metropolis-Hastings to sample from $p(D'|D)$ directly, although this might be computationally expensive.

The choice of approximation will depend on the specific form of $g$ and the complexity of the data.  Understanding the properties of $g$ is crucial for choosing an appropriate weighting scheme and assessing the validity of the resulting uncertainty estimates.  Your intuition about the behavior of $g$ for different values of $u$ is a good starting point for designing more informed weighting strategies.


Here is my procedure to approximate the uncertainty of the output of a NN conditioned on the training data $d$:
1. generate samples $u_k$, $k=1 \dots m$ ($u \in [0,1]$)
2. generate synthetic datasets $d_k \coloneqq g(d, u_k)$
3. train NNs $\hat{f}_k$ on those
4. evaluate those: $\hat{y}_k(x) = \hat{f}_k(x)$
5. weight samples such that correct distribution is obtained

I am not sure about the weighting process, which is why I come to you. I just know that I have to reweight because of the following thought experiment:
I know that $g$ gives synthetic datasets that can look completely different from another for large, but different $u$'s. The opposite is true for small but different $u$'s.
Imagine two different sampling strategies for $u_k$. The first gives very large $u$'s and the second very small ones. Since my NN learns these datasets, the same while hold for the predicted values $\hat{y}_k(x)$: First samling strategy -> high variance. Second samling strategy -> low variance. So two different uncertainty quantifications for the same dataset $d$.

Formalize my procedure step by step using exact mathematical notation.

Make sure to use consistent notation: Uppercase for random variables, f,g,h,... for functions and so on. Be extra precise and pedantic about everything.

So $D$ is the dataset RV. $d$ is a realization of it.

And I am looking for $p(\hat{Y}(x) | D)$, where $\hat{Y}(x)$ is the output of a NN (also a RV because of the dataset samples) for a $x$.

$$\mathcal{D}$$

$$\mathbf{D}$$

Uniqueness Analysis: The code explores the non-uniqueness of the retardation factor by generating numerous $R$ functions (using various methods discussed below) and simulating the resulting concentration fields. It analyzes how dissimilar R values can produce indistinguishable concentration outputs, particularly at measurement locations, highlighting challenges in using limited experimental data for inverse problems. This is shown in \verb|analyze_valid_retardations.ipynb|, \verb|news_07102024.ipynb| (random retardations section), and \verb|c_uniqueness_2.ipynb|.
Uncertainty Quantification Framework: The code develops a framework for estimating solver uncertainty using several approaches:
Modified MCMC (Markov Chain Monte Carlo): Implemented to sample the posterior distribution of R, providing a baseline UQ method (seen in \verb|news_29102024.ipynb|).
SPAN (Stochastic Perturbation Analysis of Networks): Applied to the training of the NN to explore the parameter space and assess the impact of parameter variations on the solution (shown in \verb|paper.ipynb|, \verb|news_29102024.ipynb|, and \verb|analyze_experimental_finn.ipynb|).
Data-SPAN: A novel technique combining PI3NN (Prediction Interval from 3 Neural Networks) with SPAN. PI3NN is used to generate multiple synthetic datasets from the original data. These datasets are then used to train multiple FINNs, thus capturing the uncertainty stemming from noisy/limited data (implemented in \verb|analyze_experimental_finn.ipynb|, \verb|news_29102024.ipynb|, and parts of \verb|run_exp_quantiles.ipynb|).
Full SPAN: Combining Data-SPAN and SPAN to provide a comprehensive uncertainty estimate.


% \begin{figure}
%     \centering
%     \includegraphics[width=0.9\textwidth]{figs/valid_generated_retardation_samples.png}
%     \label{fig:random_retardations}
%     \caption{Left: Retardations which result in low NMSE ( < $5 \cdot 10^{-6}$). Fat Line: Concentration-producing retardation. Middle and Right: Concentration BTC and Profile for these Retardations. Top: NMSE computed on BTC. Bottom NMSE computed on full field.}
%     %
%     % Extra details:
%     %
%     % Top:
%     % Max.  BTC MAE: 0.0007382795619641294
%     % Max. FULL MAE: 0.00842735039246223
%     % Bottom:
%     % Max.  BTC MAE: 0.00034597463041581566
%     % Max. FULL MAE: 0.00287651082264595
% \end{figure}

% As shown by figure \ref{fig:random_retardations}, there are retardation curves that look significantly different from each other but produce almost indistinguishable concentration for most spatial and temporal points. For $t=1$, the absolute difference in retardation is about $2.5$, whereas the absolute difference in concentration is less than $10^{-3}$. The reason being that $R(c)$ for some $c$ does not affect the whole $c$ field equally. This can be seen by looking at the error on the full field, which is about $10$ times as high (Max. full field MAE = $0.0084$, whereas max. BTC MAE = $0.0007$).


$$
\mathbb{1}
$$

\subsection{Motivation}
% TODO: This not good
The accurate prediction of contaminant transport in groundwater systems is crucial for effective remediation strategies and risk assessment [1, 2]. A key factor influencing contaminant migration is sorption, which is often represented by retardation factors in transport models [3, 4]. Traditional modeling approaches typically rely on predefined parametric sorption isotherms, such as linear, Freundlich, or Langmuir isotherms, to describe the relationship between dissolved and sorbed contaminant concentrations [5-7]. However, the choice and parameterization of these isotherms introduce significant conceptual uncertainty, as they may not accurately capture the complex sorption behavior observed in real-world scenarios [3].
Machine learning (ML) offers a promising alternative by enabling data-driven learning of retardation factors without relying on predefined parametric models [8-10]. The finite volume neural network (FINN) presented by \textcite{finn} combines the well-established finite volume method (FVM) with the flexibility of artificial neural networks (ANNs) to learn specific components of the governing partial differential equations (PDEs), including the retardation factor as a function of concentration [8, 11, 12]. This hybrid modeling approach allows for capturing complex sorption behavior while maintaining numerical stability and providing scientifically interpretable results [2, 8, 13].
However, like all models, ML-based predictions are inherently uncertain due to factors such as limited training data, measurement noise, and model structure [14-16]. Quantifying this uncertainty is crucial for providing reliable predictions and enabling informed decision-making in real-world applications. \textcite{finn} explores uncertainty quantification (UQ) techniques for FINN, including Bayes-by-backprop and Markov chain Monte Carlo (MCMC) methods [8, 17, 18]. These methods, while effective, can be computationally expensive, especially for complex models with a large number of parameters [15, 19, 20].

This work is motivated by the need for more efficient UQ methods for ML-based retardation factor estimation. Building upon the FINN framework, this work proposes a novel UQ approach that assumes random hyperparameters of the ANN instead of random weights, as in Bayes-by-backprop. This alternative approach aims to provide significantly faster UQ estimates while achieving comparable accuracy to existing Bayesian methods. By focusing on the efficiency of UQ without sacrificing accuracy, this work aims to make ML-based modeling of contaminant transport more accessible and applicable for real-world scenarios.

\begin{table}[h]
    \centering
    \caption{Parameter Values for Synthetic Data Generation}
    \begin{tabular}{lccr}
        \toprule
        Parameter & Symbol & Unit & Value \\
        \midrule
        \multicolumn{4}{l}{\textbf{Common parameters}} \\
        Effective diffusion coefficient & $D$ & m$^2$/day & $5.00 \times 10^{-4}$ \\
        Porosity & $\phi$ & -- & 0.29 \\
        Density & $\rho_s$ & kg/m$^3$ & 2,880 \\
        \midrule
        \multicolumn{4}{l}{\textbf{Linear isotherm}} \\
        Partitioning coefficient & $K_d$ & m$^3$/kg & $4.30 \times 10^{-4}$ \\
        \midrule
        \multicolumn{4}{l}{\textbf{Freundlich isotherm}} \\
        Freundlich's K & $K_f$ & (m$^3$/kg)$^{n_f}$ & $3.50 \times 10^{-4}$ \\
        Freundlich exponent & $n_f$ & -- & 0.87 \\
        \midrule
        \multicolumn{4}{l}{\textbf{Langmuir isotherm}} \\
        Half-concentration & $K$ & kg/m$^3$ & 1.00 \\
        Sorption capacity & $S_{max}$ & m$^3$/kg & $5.90 \times 10^{-4}$ \\
        \bottomrule
    \end{tabular}
\end{table}

------------------

# Method: Computation of Standardized Beta Coefficients

This section details the computation of standardized beta coefficients used in the analysis.

## Input Data
Let $\mathbf{X}$ be the feature matrix of size $n \times p$, where $n$ is the number of samples and $p$ is the number of features. Each element $x_{i,j}$ represents the value of feature $j$ for sample $i$. Let $\mathbf{y}$ be the target variable vector of size $n \times 1$. Each element $y_i$ represents the target value for sample $i$.


## Beta Coefficient Calculation
Add a column of ones to the standardized feature matrix $\mathbf{X}$ to account for the intercept term. The augmented matrix is denoted as $\mathbf{X}_{aug}$.
The standardized beta coefficients, including the intercept, are computed using the normal equation:
\begin{equation}
\boldsymbol{\beta} = (\mathbf{X}_{aug}^T \mathbf{X}_{aug})^{-1} \mathbf{X}_{aug}^T \mathbf{y}
\end{equation}
where:
$\boldsymbol{\beta}$ is the vector of standardized beta coefficients (size $(p+1) \times 1$).


# Method: Approximate Sensitivity Analysis from Samples

This section outlines the method used to perform approximate sensitivity analysis using provided samples.

## Input Data
Let $\mathbf{P}$ be the parameter sample matrix of size $n_{samples} \times n_{params}$, where $n_{samples}$ is the number of samples and $n_{params}$ is the number of parameters. Each element $p_{i,j}$ represents the value of parameter $j$ in sample $i$. Let $\mathbf{Y}$ be the output sample matrix of size $n_{samples} \times n_x$, where $n_x$ is the number of output variables. Each element $y_{i,k}$ represents the value of output variable $k$ in sample $i$. (If there is only one output variable, $\mathbf{Y}$ is a vector of size $n_{samples} \times 1$).

## Data Scaling
### Parameter Scaling
`StandardScaler` is applied to standardize each parameter column of $\mathbf{P}$. For each parameter $j$:
Calculate the mean: 
\begin{equation}
\mu_{p,j} = \frac{1}{n_{samples}} \sum_{i=1}^{n_{samples}} p_{i,j}
\end{equation}
Calculate the standard deviation: 
\begin{equation}
\sigma_{p,j} = \sqrt{\frac{1}{n_{samples}} \sum_{i=1}^{n_{samples}} (p_{i,j} - \mu_{p,j})^2}
\end{equation}
Scale each parameter value: 
\begin{equation}
p'_{i,j} = \frac{p_{i,j} - \mu_{p,j}}{\sigma_{p,j}}
\end{equation}
The scaled parameter matrix is $\mathbf{P}'$.

### Output Scaling (for each output dimension $k$)
Similarly, each output dimension $k$ of $\mathbf{Y}$ is standardized:
Calculate the mean: 
\begin{equation}
\mu_{y,k} = \frac{1}{n_{samples}} \sum_{i=1}^{n_{samples}} y_{i,k}
\end{equation}
Calculate the standard deviation: 
\begin{equation}
\sigma_{y,k} = \sqrt{\frac{1}{n_{samples}} \sum_{i=1}^{n_{samples}} (y_{i,k} - \mu_{y,k})^2}
\end{equation}
Scale each output value: 
\begin{equation}
y'_{i,k} = \frac{y_{i,k} - \mu_{y,k}}{\sigma_{y,k}}
\end{equation}
The scaled output vector for dimension $k$ is $\mathbf{y}'_k$.

## Linear Regression
### Overall Linear Regression (for each output dimension $k$)
A linear model is fit to the scaled data: 
\begin{equation}
\mathbf{y}'_k \approx \beta_{0,k} + \sum_{j=1}^{n_{params}} \beta_{j,k} p'_{i,j}
\end{equation}
$\beta_{j,k}$ are the regression coefficients for output dimension $k$ and parameter $j$. $\beta_{0,k}$ is the intercept.
The coefficient of determination (R-squared) is calculated: $R^2_k$.

### Individual Parameter Regression (for each output dimension $k$ and parameter $j$)
A separate linear model is fit for each parameter individually: 
\begin{equation}
\mathbf{y}'_k \approx \alpha_{0,j,k} + \alpha_{j,k} p'_{i,j}
\end{equation}
$\alpha_{j,k}$ is the regression coefficient for output dimension $k$ when using only parameter $j$.
The coefficient of determination is calculated: $R^2_{j,k}$.

## Sensitivity Index Calculation
For each output dimension $k$ and parameter $j$, the approximate sensitivity index $S_{j,k}$ is computed as:
\begin{equation}
S_{j,k} = \frac{R^2_{j,k}}{R^2_k}
\end{equation}

## Output
The sensitivity indices $S_{j,k}$ are organized into a DataFrame, where rows represent output dimensions and columns represent parameters.
